In [4]:
!pip install Pillow


  Using cached pillow-11.2.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
Using cached pillow-11.2.1-cp312-cp312-manylinux_2_28_x86_64.whl (4.6 MB)


In [3]:
from pathlib import Path
from PIL import Image, UnidentifiedImageError

def clean_invalid_images(dataset_path):
    """
    Menghapus file non-gambar dan gambar yang rusak dari direktori dataset.

    Args:
        dataset_path (str): Path ke direktori dataset.
    """
    img_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]
    dataset_path = Path(dataset_path)
    count_removed = 0

    for img_path in dataset_path.rglob("*.*"):
        # Hapus file dengan ekstensi bukan gambar
        if img_path.suffix.lower() not in img_extensions:
            print(f"❌ Bukan gambar: {img_path}")
            img_path.unlink()  # Hapus file
            count_removed += 1
        else:
            # Coba buka gambar, jika gagal berarti rusak
            try:
                with Image.open(img_path) as img:
                    img.verify()  # Validasi gambar tanpa loading penuh
            except (UnidentifiedImageError, OSError):
                print(f"⚠️ Rusak: {img_path}")
                img_path.unlink()
                count_removed += 1

    print(f"\n✅ Selesai! {count_removed} file dihapus.")

folder_dataset = "Batik Nitik Sarimbit 120"
clean_invalid_images(folder_dataset)


❌ Bukan gambar: Batik Nitik Sarimbit 120/48 Sekar Jeruk B.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/34 Kuncup Kanthil B.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/52 Sekar Blimbing B.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/27 Kawung Nitik B.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/1 Sekar Kemuning A.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/60 Sekar Gambir A.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/10 Sekar Ketongkeng A.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/56 Sekar Kepel B.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/13 Sekar Menur B.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/37 Manggar A.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/26 Sekar Nangka A.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/14 Sekar Tebu A.jpg:Zone.Identifier
❌ Bukan gambar: Batik Nitik Sarimbit 120/9 Sekar Gudh

In [6]:
from pathlib import Path
import shutil
import re

def get_class_name(file_name: str) -> str:
    """
    Mengambil nama kelas dari nama file.
    Contoh: '1 Arumdalu B 2_rotate_90.jpg' → 'Arumdalu'
    """
    # Hapus ekstensi
    name = file_name.rsplit('.', 1)[0]

    # Hapus angka label di awal dan informasi rotasi di akhir
    name = re.sub(r'^\d+\s+', '', name)                    # hapus angka awal dan spasi
    name = re.sub(r'\s+\d+(_rotate_\d+)?$', '', name)      # hapus angka urut dan rotasi

    # Hapus huruf tambahan di akhir nama kelas (misal A, B)
    name = re.sub(r'\s+[A-B]$', '', name)                  # hapus ' A' atau ' B'

    return name.strip()

def group_images_by_classname(source_folder, target_folder):
    source_folder = Path(source_folder)
    target_folder = Path(target_folder)
    target_folder.mkdir(parents=True, exist_ok=True)

    for img_path in source_folder.glob("*.jpg"):
        class_name = get_class_name(img_path.name)
        class_folder = target_folder / class_name
        class_folder.mkdir(exist_ok=True)
        
        # Cegah overwrite
        destination = class_folder / img_path.name
        if destination.exists():
            base = img_path.stem
            ext = img_path.suffix
            i = 1
            while (class_folder / f"{base}_{i}{ext}").exists():
                i += 1
            destination = class_folder / f"{base}_{i}{ext}"
        
        shutil.copy(img_path, destination)

    print("✅ Semua gambar berhasil digabungkan ke dalam folder kelas utama.")

# Contoh penggunaan:
group_images_by_classname("Batik Nitik 960", "Batik Nitik Grouped")


✅ Semua gambar berhasil digabungkan ke dalam folder kelas utama.
